In [ ]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import category_encoders as ce 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Ridge, Lasso, RidgeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from collections import Counter
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import FeatureUnion
from sklearn.impute import MissingIndicator
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV
from sklearn.inspection import permutation_importance
from category_encoders import TargetEncoder, LeaveOneOutEncoder
import random
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import KBinsDiscretizer
from functools import reduce
from sklearn.linear_model import LogisticRegression
import optuna
from optuna.samplers import RandomSampler, GridSampler, TPESampler
from sklearn.metrics import roc_auc_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neural_network import MLPClassifier

In [ ]:
BASEPATH = "../input/siim-isic-melanoma-classification"
os.chdir(BASEPATH)

In [ ]:
dftrain = pd.read_csv('train.csv') 
dftest = pd.read_csv('test.csv') 

In [ ]:
dftrain.head()

In [ ]:
dftest.head()

In [ ]:
# Create column transfomer 
def get_ct(cat_features, num_features):
    numeric_transformer1 = FeatureUnion(transformer_list=[
            ('imputer',  SimpleImputer(strategy='constant',fill_value=-1)),
            ('indicator',MissingIndicator())
            ])
    ce_cat = ce.OneHotEncoder(cols=cat_features,handle_unknown='value',handle_missing='value')
    ct1 = ColumnTransformer(
            transformers=[
                ('cat',ce_cat,cat_features),
                ('num',numeric_transformer1,num_features),
                ],remainder = 'drop')
    clf = Pipeline(steps=[('preprocessor', ct1)])
    return clf 

In [ ]:
X_train = dftrain[['sex','age_approx','anatom_site_general_challenge']]
y_train = dftrain['target']
image_names = dftrain[['image_name']]
X_test = dftest[['sex','age_approx','anatom_site_general_challenge']]
cat_features = ['sex','anatom_site_general_challenge']
num_features = ['age_approx']
ct_trans = get_ct(cat_features,num_features)
X_train = ct_trans.fit_transform(X_train)
X_test = ct_trans.transform(X_test)

In [ ]:
study_name1 = 'Ridge'
study_ridge = optuna.create_study(study_name=study_name1,direction='maximize',sampler=TPESampler(0))

# **Train Ridge**

In [ ]:
def opt_ridge(trial):    

    C = trial.suggest_loguniform('alpha',1e-7,10)
    b = int(trial.suggest_loguniform('b',1,32))
    kFold= StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
    model = RidgeClassifier(alpha=C, class_weight={0:1,1:b},random_state=0)
    kFold= StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
    scoring = 'roc_auc'
    return cross_val_score(
        model, X_train, y_train, n_jobs=-1,scoring=scoring,cv=kFold).mean()

In [ ]:
study_ridge.optimize(opt_ridge, n_trials=50)

In [ ]:
print('Total number of trials: ',len(study_ridge.trials))
trial_ridge = study_ridge.best_trial
print('Best score : {}'.format(-trial_ridge.value))
for key, value in trial_ridge.params.items():
    print("    {}: {}".format(key, value))
alpha_RID = list(trial_ridge.params.items())[0][1]
b = int(list(trial_ridge.params.items())[1][1])

# **Train LightGBM**

In [ ]:
def focal_binary_lgb(label, pred):
    def robust_pow(num_base, num_pow):
        # numpy does not permit negative numbers to fractional power
        # use this to perform the power algorithmic
        return np.sign(num_base) * (np.abs(num_base)) ** (num_pow)
    
    gamma_indct = 2.0
    # retrieve data from dtrain matrix
    #label = dtrain.label
    # compute the prediction with sigmoid
    sigmoid_pred = 1.0 / (1.0 + np.exp(-pred))
    # gradient
    # complex gradient with different parts
    g1 = sigmoid_pred * (1 - sigmoid_pred)
    g2 = label + ((-1) ** label) * sigmoid_pred
    g3 = sigmoid_pred + label - 1
    g4 = 1 - label - ((-1) ** label) * sigmoid_pred
    g5 = label + ((-1) ** label) * sigmoid_pred
    # combine the gradient
    grad = gamma_indct * g3 * robust_pow(g2, gamma_indct) * np.log(g4 + 1e-9) + \
           ((-1) ** label) * robust_pow(g5, (gamma_indct + 1))
    # combine the gradient parts to get hessian components
    hess_1 = robust_pow(g2, gamma_indct) + \
             gamma_indct * ((-1) ** label) * g3 * robust_pow(g2, (gamma_indct - 1))
    hess_2 = ((-1) ** label) * g3 * robust_pow(g2, gamma_indct) / g4
    # get the final 2nd order derivative
    hess = ((hess_1 * np.log(g4 + 1e-9) - hess_2) * gamma_indct +
            (gamma_indct + 1) * robust_pow(g5, gamma_indct)) * g1

    return grad, hess

In [ ]:
study_name2 = 'lgb'
study_lgb = optuna.create_study(study_name=study_name2,direction='maximize',sampler=TPESampler(0))

In [ ]:
def opt_lgb(trial):    

    num_leaves = int(trial.suggest_loguniform("num_leaves", 3,20))
    subsample =  trial.suggest_discrete_uniform('bfrac',0.6,1.0,q=0.1),
    subsample_freq = int(trial.suggest_discrete_uniform('bfreq',1,5,q=1.0)),
    colsample_bytree = trial.suggest_discrete_uniform('feature',0.8,1.0,q=0.05),
    reg_lambda = trial.suggest_loguniform("lambda_l2", 1, 10)
    kFold= StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
    clf  = lgb.LGBMClassifier(objective=focal_binary_lgb,random_state=0,
                                                        num_leaves = num_leaves,
                                                         subsample=subsample,subsample_freq=subsample_freq,
                                                        colsample_bytree=colsample_bytree,reg_lambda=reg_lambda)
    scoring = 'roc_auc'
    return cross_val_score(
        clf, X_train, y_train, n_jobs=-1,scoring=scoring,cv=kFold).mean()

In [ ]:
study_lgb.optimize(opt_lgb, n_trials=50)

In [ ]:
print('Total number of trials: ',len(study_lgb.trials))
trial_lgb = study_lgb.best_trial
print('Best score : {}'.format(-trial_lgb.value))
for key, value in trial_lgb.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
num_leaves = int(list(trial_lgb.params.items())[0][1])
bfrac = list(trial_lgb.params.items())[1][1]
bfreq = int(list(trial_lgb.params.items())[2][1])
feature =  list(trial_lgb.params.items())[3][1]
lambda_l2 = list(trial_lgb.params.items())[4][1]

# **Train NN**

In [ ]:
study_name3 = 'nn'
study_nn = optuna.create_study(study_name=study_name3,direction='maximize',sampler=TPESampler(0))

In [ ]:
def opt_nn(trial):    

    alpha = trial.suggest_loguniform('alpha',1e-6,10)
    z = int(trial.suggest_loguniform('z',4,32))
    kFold= StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
    model= MLPClassifier(hidden_layer_sizes=[z],alpha=alpha,random_state=0,tol=1e-4,max_iter=200)
    scoring = 'roc_auc'
    return cross_val_score(
        model, X_train, y_train, n_jobs=-1,scoring=scoring,cv=kFold).mean()

In [ ]:
study_nn.optimize(opt_nn, n_trials=25)

In [ ]:
print('Total number of trials: ',len(study_nn.trials))
trial_nn = study_nn.best_trial
print('Best score : {}'.format(-trial_nn.value))
for key, value in trial_nn.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
alpha_nn = list(trial_nn.params.items())[0][1]
z = int(list(trial_nn.params.items())[1][1])

# Make Predictions #

In [ ]:
# Finalize all models 
model_final_Ridge = RidgeClassifier(alpha=alpha_RID,class_weight={0:1,1:b},random_state=0)
model_final_LGB = lgb.LGBMClassifier(objective=focal_binary_lgb,random_state=0,
                                                        num_leaves = num_leaves,
                                                         subsample=bfrac,subsample_freq=bfreq,
                                                        colsample_bytree=feature,reg_lambda=lambda_l2)
model_final_NN = MLPClassifier(hidden_layer_sizes=[z],alpha=alpha_nn,random_state=0,tol=1e-4,max_iter=200)

# Do probability calibration for all models
# https://scikit-learn.org/stable/modules/calibration.html
kFold= StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
clf_sigmoid_Ridge = CalibratedClassifierCV(model_final_Ridge, cv=kFold, method="sigmoid")
clf_sigmoid_LGB = CalibratedClassifierCV(model_final_LGB, cv=kFold, method="sigmoid")
clf_sigmoid_NN = CalibratedClassifierCV(model_final_NN, cv=kFold, method="sigmoid") 

In [ ]:
clf_sigmoid_Ridge.fit(X_train, y_train)
clf_sigmoid_LGB.fit(X_train, y_train)
clf_sigmoid_NN.fit(X_train, y_train)

In [ ]:
# Make Predictions
y_pred_Ridge = clf_sigmoid_Ridge.predict_proba(X_test)[:,1]
y_pred_LGB = clf_sigmoid_LGB.predict_proba(X_test)[:,1]
y_pred_NN = clf_sigmoid_NN.predict_proba(X_test)[:,1]

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['target'] = (y_pred_Ridge + y_pred_LGB + y_pred_NN) / 3

In [ ]:
plt.hist(submission['target'])